In [9]:
import pandas as pd
import requests as r 
import duckdb 
from difflib import SequenceMatcher

In [2]:
with duckdb.connect('../../data/protodb.db') as conn:
    df = conn.query('select * from intel.cian').df().drop_duplicates(subset=['url'])

In [3]:
import re

In [4]:
base_search_url = 'https://dom.mingkh.ru/search?searchtype=house&address='
BASE_URL = 'https://dom.mingkh.ru'

In [5]:
df ['Дом'] = df['Дом'].fillna('').apply(lambda x: re.sub('(?!:\d+)(К)',' корпус ',x))

In [25]:
sample_ = df.loc[:,['Город','Округ','Улица','Дом']].iloc[3]

In [26]:
sample_

Город              Москва
Округ                СВАО
Улица    Сигнальный 16 ЖК
Дом                   1.1
Name: 3, dtype: object

In [37]:
' '.join(sample_.to_dict().values())

'Москва СВАО Сигнальный 16 ЖК 1.1'

In [95]:
def search_home_url(adress_dict:dict):

    home_query = ' '.join(adress_dict.values()).replace(' ','+')
    adress = adress_dict['Улица'] + ', ' + adress_dict['Дом']


    request_home_query = r.utils.quote(home_query)
    request_url = base_search_url + request_home_query


    search_table = pd.read_html(request_url,extract_links='body')[0]
    search_table[['Адрес','home_url']] = search_table['Адрес'].apply('|'.join).str.split('|',expand=True)
    search_table['match'] = search_table['Адрес'].apply(lambda x: SequenceMatcher(None,x,adress).ratio())

    
    result = search_table.set_index('home_url')['match'].idxmax()

    return result

In [219]:

def parse_home_page(url: str):
    full_url = BASE_URL + url
    _id = url.split('/')[-1]
    df_list = pd.read_html(full_url)
    home_data = pd.concat(df_list)\
        .iloc[:, :3]\
        .dropna(thresh=1)\
        .set_index(0)\
        .fillna(method='ffill', axis=1)\
        .iloc[:, 1]\
        .rename(_id)\
        .rename_axis('id')
    
    result = home_data.reset_index().drop_duplicates().pipe(
        lambda x: x.loc[x.iloc[:,0]!=x.iloc[:,1]]
    ).set_index('id')
    return result.T.assign(url=url).T.iloc[:,0]

In [243]:
def get_advanced_home_data(name_series:pd.Series):
    url_ = search_home_url(name_series)
    result = parse_home_page(url_)
    return result.to_dict()


In [254]:
def get_advanced_home_features(home_adress_df):
    result_list = {}
    for idx in home_adress_df.index:
        result_list[idx] = get_advanced_home_data(home_adress_df.fillna('').loc[idx].to_dict())

    return pd.DataFrame(result_list).T

In [255]:
new_sample = {
    'Город':'Волгоград',
    'Улица':'Еременко',
    'Дом':'102',
}

In [256]:
home_adress_df = df.loc[:,['Город','Округ','Улица','Дом']]

In [257]:
zz = get_advanced_home_features(home_adress_df.iloc[:5])

In [258]:
zz

,Год ввода в эксплуатацию,Дом признан аварийным,Состояние дома,Количество квартир,Количество нежилых помещений,Количество балконов,Количество лоджий,Класс энергетической эффективности,Количество подъездов,Количество лифтов,...,Вентиляция,Система водостоков,Газоснабжение,Система пожаротушения,Электроснабжение,Мусоропровод,"Площадь подвала, кв.м",Год проведения последнего капитального ремонта,Тип системы газоснабжения,Количество вводов системы газоснабжения
0,2014,Нет,Исправный,283,1,72,304,С (Нормальный),4,8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020,Нет,Исправный,1303,35,NaN,NaN,NaN,3,17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1957,Нет,Исправный,60,8,48,NaN,D (Пониженный),3,3,...,Вытяжная вентиляция,Внутренние водостоки,Центральное,Отсутствует,Центральное,Отсутствует,1103,2020,центральное,1 единиц
3,1979,Нет,Исправный,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021,Нет,Исправный,592,61,NaN,NaN,NaN,9,18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
